__[Based on A novel architecture for web-based attack detection using convolutional neural network](https://www.sciencedirect.com/science/article/pii/S0167404820303692?via%3Dihub)__

<img src="https://miro.medium.com/max/640/1*ulfFYH5HbWpLTIfuebj5mQ.gif" width = 400/>

>## PREPROCESSING
 - __The preprocessing consists of two steps:__ 
   1. The production of dictionaries
   2. The production of matrixes

In [ ]:
import pandas as pd

data = pd.read_csv('normalTrafficTraining.txt',sep="/n/n", header= None)

print("Shape:", data.shape)
print("\nRecords Example:\n", data.head()) 
print("\nInfo:\n", data.describe())

***
__`The header parts of the HTTP requests are removed. Since the header 
portions are generally the same in the HTTP requests dataset, they reduce the size of the data 
due to reduction of unnecessary data and eliminate the duplication of the data in datasets. The 
URL and payload portions of the HTTP request in datasets are selected`__
***

In [ ]:
whData = data[data[0].str.contains('GET|POST') 
              | (data[0].str.contains(': ') == False)]

whData.loc[:, 0] = whData[0].apply(lambda x: 
    x.replace("GET ", "").replace("POST ", "").replace(" HTTP/1.1", ""))

whData.loc[:, 0] = whData[0].apply(lambda x: x.strip())

In [ ]:
print(whData.shape)
print(whData.head(10))

In [ ]:
payload = whData[0].str.split("&",
                              expand=True).stack().reset_index(level=1,
                                                               drop=True)
payload = whData[0].str.split("?",
                              expand=True).stack().reset_index(level=1,
                                                               drop=True)
payload.head()

>**Numpy arrays. For further investigation**

In [ ]:
import numpy as np
npdata = data.to_numpy()

for i in range(0,20):
    print(npdata[i])

In [ ]:
npwhdata = whData.to_numpy()
for i in range(0,10):
    print(npwhdata[i])

In [ ]:
npPayload = payload.to_numpy()
for i in range(0,10):
    print(npPayload[i])

>### **Results**
`Based on normalTrafficTraining.txt we've got the following`
-  __Dataframes:__
    *  data         _(source data)_
    *  whData       _(contains URL and Payload)_
    *  __payload__      _(Splitted Payload)_
>
- __Numpy Arrays:__
    *  npdata       _(source data)_
    *  npwhdata     _(contains URL and Payload)_
    *  __npPayload__    _(Splitted Payload)_

>

***
## Dictionary production scheme

<img src="dictionary production.png"/>

***

***

In [ ]:
# Define some raw data containing URLs and payloads (variables and values)
data = [
    {"url": "http://example.com/page1?var1=val1&var2=val2", "payloads": ["var1=val1", "var2=val2"]},
    {"url": "http://example.com/page2?var1=val1&var2=val2", "payloads": ["var1=val1", "var2=val2"]},
    {"url": "http://example.com/page3?var1=val1&var2=val3", "payloads": ["var1=val1", "var2=val3"]},
]

# Create dictionaries from the raw data
payload_dict = {}
for dat in data:
    # Add the payloads to the payload dictionary
    for payload in dat["payloads"]:
        if payload not in payload_dict:
            payload_dict[payload] = 1
        else:
            payload_dict[payload] += 1

# Print the dictionaries
print(payload_dict)

# Use the Bag-of-Words technique to create matrixes from the dictionaries
payload_matrix = []
for dat in data:
    # Create a BoW vector for the payloads
    payload_vector = [0] * len


***

In [ ]:
# Define some raw data containing URLs and payloads
data = [
    {"url": "http://example.com/page1?var1=val1&var2=val2", "payload": "Hello world!"},
    {"url": "http://example.com/page2?var1=val1&var2=val2", "payload": "Hello world!"},
    {"url": "http://example.com/page3?var1=val1&var2=val2", "payload": "Goodbye world!"},
]

# Create dictionaries from the raw data
url_dict = {}
payload_dict = {}
for dat in data:
    # Split the URL into variables and values using the filter
    url_parts = dat["url"].split("?")[1].split("&")
    for part in url_parts:
        variable, value = part.split("=")
        if variable not in url_dict:
            url_dict[variable] = [value]
        else:
            url_dict[variable].append(value)
    # Add the payload to the payload dictionary
    if dat["payload"] not in payload_dict:
        payload_dict[dat["payload"]] = 1
    else:
        payload_dict[dat["payload"]] += 1

# Print the dictionaries
print(url_dict)
print(payload_dict)

# Use the Bag-of-Words technique to create matrixes from the dictionaries
url_matrix = []
payload_matrix = []
for dat in data:
    # Create a BoW vector for the URL
    url_vector = [0] * len(url_dict)
    url_parts = dat["url"].split("?")[1].split("&")
    for part in url_parts:
        variable, value = part.split("=")
        if variable in url_dict:
            url_vector[url_dict[variable].index(value)] += 1
    url_matrix.append(url_vector)
    # Create a BoW vector for the payload
    payload_vector = [0] * len(payload_dict)
    if dat["payload"] in payload_dict:
        payload_vector[list(payload_dict.keys()).index(dat["payload"])] += 1
    payload_matrix.append(payload_vector)

# Print the matrixes
print(url_matrix)
print(payload_matrix)


In [ ]:
#cool tools)
import nltk
import re

In [ ]:
#demonstration of tokenization
txt = 'id=3&nombre=Vino+Rioja&precio=100&cantidad=55&B1=A%F1adir+al+carrito'
nltk.tokenize.regexp_tokenize(txt, pattern=r"&", gaps=True)

In [ ]:
#demonstration no.2
dataset = ['Game of Thrones is an amazing tv series!',
'Game of Thrones is the best tv series!',
'Game of Thrones is so great']

word2count = {}
for data1 in dataset:
    words = nltk.tokenize.regexp_tokenize(data1, pattern=r"\s", gaps=True)
    for word in words:
        if word not in word2count.keys():
            word2count[word] = 1
        else:
            word2count[word] += 1

print(word2count)

In [ ]:
#Do procesing only with payloads (without URLs) because I still have no idea why we have URL here))
payload_v2 = data[data[0].str.contains('GET|POST') 
              | (data[0].str.contains(': ') == False)]
payload_v2 = payload_v2[payload_v2[0].str.contains('http') == False]

dict = {}
for rdata in payload_v2[0]:
    words = nltk.tokenize.regexp_tokenize(rdata, pattern=r"&", gaps=True)
    for word in words:
        if word not in dict.keys():
            dict[word] = 1
        else:
            dict[word] += 1


In [ ]:
#try at your own risk
print(dict)

***